#### Pulled from : https://www.tensorflow.org/tutorials/quickstart/beginner

## Uses Keras (https://www.tensorflow.org/guide/keras/overview)

1. Load prebuilt dataset
2. Build neural network machine
3. Train neural network
4. Evaluate the machine

## Setup tensorflow

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Using keras sequential (https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

<a href="https://www.tensorflow.org/guide/keras/sequential_model">Sequential</a> is useful for stacking layers where each layer has one input <a href="https://www.tensorflow.org/guide/tensor">tensor</a> and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten">Flatten</a>, <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense">Dense</a>, and <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout">Dropout</a> layers.

For each example, the model returns a vector of <a href="https://developers.google.com/machine-learning/glossary#logits">logits</a> or <a href="https://developers.google.com/machine-learning/glossary#log-odds">log-odds</a> scores, one for each class.

In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.08824034, -0.26079616, -0.07559998,  0.2138725 ,  0.1205364 ,
        -0.33668992, -0.37281877, -0.14891198, -0.19410911,  0.1228098 ]],
      dtype=float32)

The <a href="https://www.tensorflow.org/api_docs/python/tf/nn/softmax">tf.nn.softmax</a> function converts these logits to probabilities for each class: 

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.09953116, 0.08375659, 0.10079726, 0.13463718, 0.1226393 ,
        0.07763521, 0.07488041, 0.09367198, 0.08953252, 0.12291842]],
      dtype=float32)

Define a loss function for training using <a href="https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy">losses.SparseCategoricalCrossentropy</a>:

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)